# Base study

This file contains code to run the evaluation for the base study for all instances

Evaluation runs:
- Baseline with Perses & Vulcan
- SeRu with Perses & Vulcan

## Setup

In [1]:
%useLatestDescriptors
%use coroutines

In [11]:
import java.io.File
import kotlin.io.path.Path
import kotlin.io.path.listDirectoryEntries
import kotlin.io.path.walk
import java.io.IOException
import java.util.concurrent.TimeUnit

val ignoredIssues = listOf("issue_2", "issue_2490")
val instances = File("instances")
    .walk()
    .filter { it.isDirectory && (it.name.startsWith("f") || it.name.startsWith("v")) }
    .filter { ignoredIssues.all { issue -> !it.parent.endsWith(issue) } }
    .map { it.toPath() }

data class CommandOutput(val stdout: String, val stderr: String)

fun String.runCommand(workingDir: File): CommandOutput? = runBlocking(Dispatchers.IO) {
    withTimeout(4.hours) {
        val proc = ProcessBuilder(*split(" ").toTypedArray()).directory(workingDir).start()

        try {
            val stdout = async { runInterruptible { proc.inputReader().use { it.readText() } } }
            val stderr = async { runInterruptible { proc.errorReader().use { it.readText() } } }

            runInterruptible { proc.waitFor() }

            CommandOutput(stdout = stdout.await(), stderr = stderr.await())
        } catch (e: IOException) {
            e.printStackTrace()
            null
        } finally {
            proc.destroy()
        }
    }
}

fun String.runCommandInRoot() = runCommand(File(".."))

## Compile SeRu

In [12]:
"go generate ./...".runCommandInRoot()
"go build".runCommandInRoot()

CommandOutput(stdout=, stderr=)

## Run evaluation

### Setup

In [13]:
import java.nio.file.Path
import kotlin.io.path.div
import kotlin.io.path.pathString

val Path.firstCueFile get() = this / listDirectoryEntries(glob = "*.cue").first().fileName
val Path.firstScriptFile get() = this / listDirectoryEntries(glob = "*.sh").first().fileName
fun seruBaseCommand(studyDir: Path, instance: Path) = "./seru -i ${studyDir / instance.firstCueFile} -t ${studyDir / instance.firstScriptFile} -m" 

fun runSeruPersesIn(studyDir: Path, instance: Path) = seruBaseCommand(studyDir, instance).runCommandInRoot()
fun runSeruVulcanIn(studyDir: Path, instance: Path) = "${seruBaseCommand(studyDir, instance)} -r vulcan".runCommandInRoot()

In [14]:
import kotlin.io.path.*

val studyPath = Path("study")

fun moveLatestSeruResultTo(dir: Path) = Path("..")
    .listDirectoryEntries(glob = "seru*")
    .maxBy { it.getLastModifiedTime() }
    .moveTo(dir / "output")

fun saveLogsIfExisting(dir: Path, logs: CommandOutput?) = logs?.let {
    (dir / "log.txt").writeBytes(it.stdout.toByteArray())
    (dir / "err.txt").writeBytes(it.stderr.toByteArray())
}

// Perses creates a backup file of the original input
fun Path.removePersesBackupFile() = listDirectoryEntries(glob = "*.orig")
    .firstOrNull()
    ?.deleteExisting()

fun Sequence<Path>.runOnInstances(outputDir: String, runsPerInstance: Int, runCmd: (Path, Path) -> CommandOutput?) {
    val totalRuns = toList().size * runsPerInstance
    forEachIndexed { index, instance ->
        val instanceOutputPath = Path(outputDir) / instance
        repeat(runsPerInstance) {
            val instanceDir = (instanceOutputPath / "run_$it").createDirectories()
            val cur = it + (index * runsPerInstance) + 1
            println("$cur/$totalRuns - Start run for $instanceDir")

            val output = runCmd(studyPath, instance)

            moveLatestSeruResultTo(instanceDir)
            saveLogsIfExisting(instanceDir, output)
            instance.removePersesBackupFile()
        }
    }
}

### Run all above to start with evaluation

#### SeRu + Perses (all instances)

Takes around 45m (M1 MacBook Pro)

In [6]:
instances.runOnInstances(outputDir = "results_perses",runsPerInstance = 5, runCmd = ::runSeruPersesIn)

1/80 - Start run for /Users/markus/Documents/UniProjects/MA/seru/study/results_perses/instances/extra/issue_2_inlined/final/run_0
2/80 - Start run for results_perses/instances/extra/issue_2_inlined/final/run_1
3/80 - Start run for results_perses/instances/extra/issue_2_inlined/final/run_2
4/80 - Start run for results_perses/instances/extra/issue_2_inlined/final/run_3
5/80 - Start run for results_perses/instances/extra/issue_2_inlined/final/run_4
6/80 - Start run for /Users/markus/Documents/UniProjects/MA/seru/study/results_perses/instances/panic/issue_2584/v1/run_0
7/80 - Start run for results_perses/instances/panic/issue_2584/v1/run_1
8/80 - Start run for results_perses/instances/panic/issue_2584/v1/run_2
9/80 - Start run for results_perses/instances/panic/issue_2584/v1/run_3
10/80 - Start run for results_perses/instances/panic/issue_2584/v1/run_4
11/80 - Start run for /Users/markus/Documents/UniProjects/MA/seru/study/results_perses/instances/panic/issue_2584/v2/run_0
12/80 - Start ru

#### SeRu + Vulcan (all instances)

Takes several 12+ hours, with some timeouts

Does not run instance `issue_2246/v1` and `issue_2209/v1` since they timeout after 2h (and even 5h)

In [6]:
val timeoutInstances = listOf("issue_2246/v1", "issue_2209/v1")
    .joinToString(separator = "|", prefix = "(", postfix = ")")

In [8]:
instances
    .filterNot { it.pathString.matches("^.*$timeoutInstances$".toRegex()) }
    .filter { it.endsWith("issue_2490_inlined/v1") }
    .runOnInstances(outputDir = "results_vulcan", runsPerInstance = 5, runCmd = ::runSeruVulcanIn)

1/5 - Start run for /Users/markus/Documents/UniProjects/MA/seru/study/results_vulcan/instances/panic/issue_2490_inlined/v1/run_0
2/5 - Start run for results_vulcan/instances/panic/issue_2490_inlined/v1/run_1
3/5 - Start run for results_vulcan/instances/panic/issue_2490_inlined/v1/run_2
4/5 - Start run for results_vulcan/instances/panic/issue_2490_inlined/v1/run_3
5/5 - Start run for results_vulcan/instances/panic/issue_2490_inlined/v1/run_4


### Ground truth

In [9]:
val persesJar = "perses_deploy.jar"
val cueJar = "cue.jar"
val tempDir = "./seru_perses"
val statFile = "$tempDir/stats"

#### Perses
Takes around 10m

In [22]:
instances.runOnInstances(outputDir = "groundtruth_perses", runsPerInstance = 5) { studyDir, instanceDir ->
    buildString {
        append("java -jar $persesJar")
        append(" -i ${studyDir / instanceDir.firstCueFile}")
        append(" -t ${studyDir / instanceDir.firstScriptFile}")
        append(" -o $tempDir")
        append(" --stat-dump-file $statFile")
        append(" --call-formatter true")
        append(" --language-ext-jars $cueJar")
    }.runCommandInRoot()
}

1/80 - Start run for /Users/markus/Documents/UniProjects/MA/seru/study/groundtruth_perses/instances/extra/issue_2_inlined/final/run_0
2/80 - Start run for groundtruth_perses/instances/extra/issue_2_inlined/final/run_1
3/80 - Start run for groundtruth_perses/instances/extra/issue_2_inlined/final/run_2
4/80 - Start run for groundtruth_perses/instances/extra/issue_2_inlined/final/run_3
5/80 - Start run for groundtruth_perses/instances/extra/issue_2_inlined/final/run_4
6/80 - Start run for /Users/markus/Documents/UniProjects/MA/seru/study/groundtruth_perses/instances/panic/issue_2584/v1/run_0
7/80 - Start run for groundtruth_perses/instances/panic/issue_2584/v1/run_1
8/80 - Start run for groundtruth_perses/instances/panic/issue_2584/v1/run_2
9/80 - Start run for groundtruth_perses/instances/panic/issue_2584/v1/run_3
10/80 - Start run for groundtruth_perses/instances/panic/issue_2584/v1/run_4
11/80 - Start run for /Users/markus/Documents/UniProjects/MA/seru/study/groundtruth_perses/instance

#### Vulcan

In [15]:
instances
    .filterNot { it.pathString.matches("^.*$timeoutInstances$".toRegex()) }
    .filter { it.endsWith("issue_2473/v1") }
    .runOnInstances(outputDir = "groundtruth_vulcan", runsPerInstance = 5) { studyDir, instanceDir ->
    buildString {
        append("java -jar $persesJar")
        append(" -i ${studyDir / instanceDir.firstCueFile}")
        append(" -t ${studyDir / instanceDir.firstScriptFile}")
        append(" -o $tempDir")
        append(" --stat-dump-file $statFile")
        append(" --call-formatter true")
        append(" --language-ext-jars $cueJar")
        append(" --enable-vulcan true")
    }.runCommandInRoot()
}

1/5 - Start run for /Users/markus/Documents/UniProjects/MA/seru/study/groundtruth_vulcan/instances/error/issue_2473/v1/run_0
2/5 - Start run for groundtruth_vulcan/instances/error/issue_2473/v1/run_1
3/5 - Start run for groundtruth_vulcan/instances/error/issue_2473/v1/run_2
4/5 - Start run for groundtruth_vulcan/instances/error/issue_2473/v1/run_3
5/5 - Start run for groundtruth_vulcan/instances/error/issue_2473/v1/run_4


## Ablation study

Takes around 2h30m

In [7]:
fun ablationPersesCmd(studyDir: Path, instanceDir: Path, strategy: Int) =
    "${seruBaseCommand(studyDir, instanceDir)} --active-strategies $strategy".runCommandInRoot()

val totalStrategies = 14

In [6]:

repeat(totalStrategies) { strategyIndex ->
    println("Active strategy: $strategyIndex")
    instances.runOnInstances("ablation_results/strategy_$strategyIndex", runsPerInstance = 5) { studyDir, instanceDir ->
        ablationPersesCmd(studyDir, instanceDir, strategyIndex)
    }
}

Active strategy: 0
1/80 - Start run for ablation_results/strategy_0/instances/extra/issue_2_inlined/final/run_0


org.jetbrains.kotlinx.jupyter.exceptions.ReplInterruptedException: The execution was interrupted

#### All except constant propagation

In [9]:
val constantPropagationIndex = 8
val strategiesArg = (0..<totalStrategies).filter { it != constantPropagationIndex }.joinToString(",")
strategiesArg

0,1,2,3,4,5,6,7,9,10,11,12,13

In [10]:
instances.runOnInstances("ablation_results/no_constant_propagation", runsPerInstance = 5) { studyDir, instanceDir -> 
    "${seruBaseCommand(studyDir, instanceDir)} --active-strategies $strategiesArg".runCommandInRoot()
}

1/80 - Start run for /Users/markus/Documents/UniProjects/MA/seru/study/ablation_results/no_constant_propagation/instances/extra/issue_2_inlined/final/run_0
2/80 - Start run for ablation_results/no_constant_propagation/instances/extra/issue_2_inlined/final/run_1
3/80 - Start run for ablation_results/no_constant_propagation/instances/extra/issue_2_inlined/final/run_2
4/80 - Start run for ablation_results/no_constant_propagation/instances/extra/issue_2_inlined/final/run_3
5/80 - Start run for ablation_results/no_constant_propagation/instances/extra/issue_2_inlined/final/run_4
6/80 - Start run for /Users/markus/Documents/UniProjects/MA/seru/study/ablation_results/no_constant_propagation/instances/panic/issue_2584/v1/run_0
7/80 - Start run for ablation_results/no_constant_propagation/instances/panic/issue_2584/v1/run_1
8/80 - Start run for ablation_results/no_constant_propagation/instances/panic/issue_2584/v1/run_2
9/80 - Start run for ablation_results/no_constant_propagation/instances/pani